In [ ]:
from queue import Queue

import matplotlib.pyplot as plt
import tracemalloc as tm
import numpy as np
import pandas as pd
import time
from threading import Thread


def gen_dataset_v2(size: int):
    max_value = size * 10
    dataset = np.random.randint(0, max_value, size)
    tsum = np.random.choice(dataset, size=int(size / 3))
    return dataset, sum(tsum) + np.random.randint(0, max_value), max_value


def accurate_algorithm(dataset: np.ndarray, sum: int):
    dataset = dataset.copy()

    def check(dataset: np.ndarray, sum: int, n: int):
        if sum == 0:
            return True
        elif n == 0 and sum != 0:
            return False
        elif dataset[n - 1] > sum:
            return check(dataset, sum, n - 1)
        return check(dataset, sum, n - 1) or check(dataset, sum - dataset[n - 1], n - 1)

    return check(dataset, sum, dataset.size)


def dynamic_algorithm(dataset: np.ndarray, sum: int, if_print=False):
    dataset = dataset.copy()
    dataset.sort()

    def print_subset(t_dataset: np.ndarray, t_subset: np.ndarray, t_sum):
        print(" ")
        print("\t\tsubsum")
        print('number|| ', end="")
        for t_sub_sum in range(0, t_sum + 1):
            print("{0:4d} |".format(t_sub_sum), end="")
        print()
        print('    x || ', end="")
        for t_sub_sum in range(0, t_sum + 1):
            if t_subset[0, t_sub_sum]:
                print("   T |", end="")
            else:
                print("   F |", end="")
        print()

        for t_number_id in range(0, len(t_dataset)):
            print(' {0:4d} || '.format(dataset[t_number_id]), end="")
            for t_sub_sum in range(0, t_sum + 1):
                if t_subset[t_number_id + 1, t_sub_sum]:
                    print("   T |", end="")
                else:
                    print("   F |", end="")
            print()

    subset = np.full((dataset.size + 1, sum + 1), False, dtype=bool)
    subset[:, 0] = True
    set_size = len(dataset)

    for number_id in range(1, len(dataset) + 1):
        if dataset[number_id - 1] <= sum:
            for subsum in range(1, sum + 1):
                if subsum < dataset[number_id - 1]:
                    subset[number_id, subsum] = subset[number_id - 1, subsum]
                else:
                    subset[number_id, subsum] = subset[number_id - 1, subsum] or \
                                                subset[number_id - 1, subsum - dataset[number_id - 1]]
                if subsum == sum and subset[number_id, subsum]:
                    return True

    if if_print:
        print_subset(dataset, subset, sum)

    return subset[set_size, sum]


def if_correct(dataset, sum):
    return dynamic_algorithm(dataset, sum, False) == accurate_algorithm(dataset, sum)


def get_stat_with_threads_and_batch(n: int, tsizes, batch):
    results = []
    for x in range(0, int(n / batch)):
        print("Start batch: "+str(x))
        results.append(get_stat_with_threads(batch, tsizes))
    if n % batch > 0:
        results.append(get_stat_with_threads(n % batch, tsizes))
    return pd.concat(results, ignore_index=True)


def get_stat_with_threads(n: int, tsizes):
    que = Queue()
    threads_list = list()

    for x in range(n):
        t = Thread(target=lambda q, arg1: q.put(get_stats(arg1)),
                   args=(que, tsizes.copy()))
        t.start()
        threads_list.append(t)

    # Join all the threads
    counter = 0
    for t in threads_list:
        t.join()
        print("proces: " + str(counter) + "finished")
        counter = counter + 1
    # Check thread's return value
    results = []
    while not que.empty():
        results.append(que.get())
    return pd.concat(results, ignore_index=True)


def get_stats(tsizes):
    data_time = []
    data_mem = []
    data_correct = []
    data_size = []
    data_max_value = []
    data_sum = []
    data_algorithm = []
    for tsize in tsizes:
        tdataset, tsum, tmax_value = gen_dataset_v2(tsize)
        for talgorithm in ["accurate", "dynamic"]:
            ttime, tmem, tresult, tcorrect = get_time_mem_correct_result(tdataset, tsum, talgorithm)
            if ttime > 1.0:
                print("t:{0:4f} |n:{1} s:{2} a:{3} ".format(ttime, tsize, tsum, talgorithm))
            # print("t:{0:4f} |".format(ttime))
            data_time.append(ttime)
            data_mem.append(tmem)
            data_correct.append(tcorrect)
            data_size.append(tsize)
            data_max_value.append(tmax_value)
            data_sum.append(tsum)
            data_algorithm.append(talgorithm)
    return pd.DataFrame({
        "time": data_time,
        "mem": data_mem,
        "correct": data_correct,
        "size": data_size,
        "max_value": data_max_value,
        "sum": data_sum,
        "algorithm": data_algorithm
    })


def get_time_mem_correct_result(dataset, tsum, talgorithm):
    tm.start()
    start_time = time.time()
    if talgorithm == "accurate":
        result = accurate_algorithm(dataset, tsum)
    else:
        result = dynamic_algorithm(dataset, tsum, False)
    end_time = time.time()
    mem_use = tm.get_traced_memory()[1]
    tm.stop()
    ttime = end_time - start_time
    correct = True if talgorithm == "accurate" else if_correct(dataset, tsum)
    return ttime, mem_use, result, correct


def gen_plot(x_label, y_label, dataframe):
    plot_data_acc = dataframe[[x_label, y_label]].where(dataframe["algorithm"] == "accurate")\
        .groupby(x_label).mean()
    plot_data_dyn = dataframe[[x_label, y_label]].where(dataframe["algorithm"] == "dynamic").groupby(x_label).mean()
    plt.plot(
        plot_data_acc.index,
        plot_data_acc[y_label],
        "r-",
        label="Accuracy"
    )
    plt.plot(
        plot_data_dyn.index,
        plot_data_dyn[y_label],
        "b-",
        label="Dynamic"
    )
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    plt.grid(True, color='0.95')
    plt.legend(title='Parameter where:')
    plt.title('Plot ' + x_label + ' / ' + y_label)
    plt.savefig('Plot_' + x_label + '-' + y_label + ".png")
    plt.show()



df = get_stat_with_threads_and_batch(
    n=10,
    tsizes=[5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70],
    batch=10
)

print(df)

gen_plot("size", "correct", df)
gen_plot("size", "time", df)
gen_plot("size", "mem", df)
gen_plot("max_value", "time", df)
gen_plot("max_value", "mem", df)
gen_plot("sum", "time", df)
gen_plot("sum", "mem", df)

Start batch: 0


In [ ]:
def gen_plot(x_label, y_label, dataframe):

    def get_smooth(X,Y):
        from scipy.interpolate import make_interp_spline, BSpline
        xnew = np.linspace(X.min(), X.max(), 300)
        spl = make_interp_spline(X, Y, k=3)  # type: BSpline
        ynew = spl(xnew)
        return xnew, ynew
    plot_data_acc = dataframe[[x_label, y_label]].where(dataframe["algorithm"] == "accurate")\
        .groupby(x_label).mean()
    plot_data_dyn = dataframe[[x_label, y_label]].where(dataframe["algorithm"] == "dynamic").groupby(x_label).mean()

    xnew,ynew = get_smooth(plot_data_acc.index,plot_data_acc[y_label])
    plt.plot(
        xnew,
        ynew,
        "r-",
        label="Accuracy smooth"
    )
    plt.plot(
        plot_data_acc.index,
        plot_data_acc[y_label],
        "r*",
        label="Accuracy"
    )

    xnew,ynew = get_smooth(plot_data_dyn.index,plot_data_dyn[y_label])
    plt.plot(
        xnew,
        ynew,
        "b-",
        label="Dynamic smooth"
    )
    plt.plot(
        plot_data_dyn.index,
        plot_data_dyn[y_label],
        "b*",
        label="Dynamic"
    )
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    plt.grid(True, color='0.95')
    plt.legend(title='Parameter where:')
    plt.title('Plot ' + x_label + ' / ' + y_label)
    plt.savefig('Plot_' + x_label + '-' + y_label + ".png")
    plt.show()

gen_plot("size", "correct", df)
gen_plot("size", "time", df)
gen_plot("size", "mem", df)
gen_plot("max_value", "time", df)
gen_plot("max_value", "mem", df)
gen_plot("sum", "time", df)
gen_plot("sum", "mem", df)
